In [1]:
from scipy.stats import skew
from IPython.display import display
import matplotlib.pyplot as plt
import sys
sys.path.append('/Users/yanhuan/anaconda/pkgs/seaborn-0.7.1-py27_0/lib/python2.7/site-packages')
import seaborn as sns
import numpy as np
import pandas as pd


SP500=pd.read_csv('SP500_new.csv')
Nasdaq=pd.read_csv('Nasdaq_new.csv')
DJI=pd.read_csv('DJI_new.csv')
DAX=pd.read_csv('DAX_new.csv')
Paris=pd.read_csv('Paris_new.csv')
Tokyo=pd.read_csv('Tokyo_new.csv')
HongKong=pd.read_csv('HongKong_new.csv')
Aus=pd.read_csv('Aus_new.csv')

Data Preprocess

In [2]:
Target=SP500['Adj Close'].pct_change(1).shift(-1)

In [3]:
datasets=[SP500,Nasdaq,DJI,DAX,Paris,Tokyo,HongKong,Aus]
namesets=['SP500','Nasdaq','DJI','DAX','Paris','Tokyo','HongKong','Aus']

this fuction is used for feature engineering,to be specific,make Multiple Day Returns,Returns Moving Average and Time Lagged Returns. 

In [4]:
def addFeatures(datasets,DaysRet,DaysRetMovAvg,DaysLags):
    """
    compute dataset's DaysReturns,DaysReturnMovAvg and DaysLags
    DaysReturnMovAvg must >=2
    """
    Max=max(DaysRet,DaysRetMovAvg,DaysLags+1)
    for i in range(len(datasets)):
        dataset=datasets[i]
        nameset=namesets[i]
        for i in range(1,DaysRet+1):
            dataset[nameset+'_'+str(i)+'_'+'DaysRet'] = dataset['Adj Close'].pct_change(i)
        for i in range(2,DaysRetMovAvg+1):
            dataset[nameset+'_'+str(i)+'_'+'DaysRetMovAvg'] = pd.rolling_mean(dataset[nameset+'_'+str(1)+'_'+'DaysRet'], i)
        for i in range(1,DaysLags+1):
            dataset[nameset+'_'+str(i)+'_'+'DaysLags'] = dataset[nameset+'_'+str(1)+'_'+'DaysRet'].shift(i)
        dataset=dataset[Max:]
    return Max

In [5]:
Max=addFeatures(datasets,9,9,9)

/Users/yanhuan/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:13: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=2,center=False).mean()
  del sys.path[0]
/Users/yanhuan/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:13: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=3,center=False).mean()
  del sys.path[0]
/Users/yanhuan/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:13: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=4,center=False).mean()
  del sys.path[0]
/Users/yanhuan/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:13: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, re

In [6]:
def mergedataset(datasets):
    useless=['Date','Open','High','Low','Close','Adj Close','Volume']
    for i in range(len(datasets)):
        datasets[i]=datasets[i].drop(useless, axis = 1)
    new_dataset = pd.concat(datasets,axis=1)
    return new_dataset

In [7]:
features=mergedataset(datasets)

In [8]:
features=features[Max:-1]
Target=Target[Max:-1]

In [9]:
features.head()

,SP500_1_DaysRet,SP500_2_DaysRet,SP500_3_DaysRet,SP500_4_DaysRet,SP500_5_DaysRet,SP500_6_DaysRet,SP500_7_DaysRet,SP500_8_DaysRet,SP500_9_DaysRet,SP500_2_DaysRetMovAvg,...,Aus_9_DaysRetMovAvg,Aus_1_DaysLags,Aus_2_DaysLags,Aus_3_DaysLags,Aus_4_DaysLags,Aus_5_DaysLags,Aus_6_DaysLags,Aus_7_DaysLags,Aus_8_DaysLags,Aus_9_DaysLags
10,0.008036,0.009905,0.013110,0.020076,0.007630,0.014865,0.009641,0.024037,0.038977,0.004945,...,-0.003458,-0.010261,0.002011,-0.000455,0.005798,-0.012021,-0.010294,-0.001675,0.009623,-0.001689
11,0.011699,0.019828,0.021720,0.024962,0.032010,0.019418,0.026738,0.021452,0.036017,0.009867,...,-0.004062,-0.013848,-0.010261,0.002011,-0.000455,0.005798,-0.012021,-0.010294,-0.001675,0.009623
12,-0.005317,0.006319,0.014406,0.016287,0.019512,0.026522,0.013997,0.021278,0.016021,0.003191,...,-0.002841,0.004190,-0.013848,-0.010261,0.002011,-0.000455,0.005798,-0.012021,-0.010294,-0.001675
13,-0.017507,-0.022731,-0.011299,-0.003354,-0.001505,0.001663,0.008551,-0.003755,0.003398,-0.011412,...,-0.001098,0.009310,0.004190,-0.013848,-0.010261,0.002011,-0.000455,0.005798,-0.012021,-0.010294
14,-0.002578,-0.020041,-0.025251,-0.013848,-0.005923,-0.004080,-0.000919,0.005950,-0.006324,-0.010043,...,-0.001073,0.005397,0.009310,0.004190,-0.013848,-0.010261,0.002011,-0.000455,0.005798,-0.012021


In [10]:
features.tail()

,SP500_1_DaysRet,SP500_2_DaysRet,SP500_3_DaysRet,SP500_4_DaysRet,SP500_5_DaysRet,SP500_6_DaysRet,SP500_7_DaysRet,SP500_8_DaysRet,SP500_9_DaysRet,SP500_2_DaysRetMovAvg,...,Aus_9_DaysRetMovAvg,Aus_1_DaysLags,Aus_2_DaysLags,Aus_3_DaysLags,Aus_4_DaysLags,Aus_5_DaysLags,Aus_6_DaysLags,Aus_7_DaysLags,Aus_8_DaysLags,Aus_9_DaysLags
5026,-0.006697,0.001595,0.001879,-0.000365,-0.001360,0.003145,0.002163,0.001331,0.001599,0.000825,...,0.001788,0.005404,0.001874,-0.012119,0.010584,0.016732,0.000000,0.000211,0.001659,-0.000053
5027,-0.000583,-0.007275,0.001011,0.001295,-0.000947,-0.001942,0.002560,0.001579,0.000748,-0.003640,...,-0.000164,-0.008251,0.005404,0.001874,-0.012119,0.010584,0.016732,0.000000,0.000211,0.001659
5028,-0.000456,-0.001038,-0.007728,0.000555,0.000839,-0.001403,-0.002397,0.002103,0.001123,-0.000519,...,0.000603,-0.015910,-0.008251,0.005404,0.001874,-0.012119,0.010584,0.016732,0.000000,0.000211
5029,0.001561,0.001104,0.000521,-0.006179,0.002117,0.002401,0.000156,-0.000840,0.003668,0.000553,...,0.000796,0.007113,-0.015910,-0.008251,0.005404,0.001874,-0.012119,0.010584,0.016732,0.000000
5030,0.000316,0.001877,0.001421,0.000837,-0.005865,0.002433,0.002717,0.000472,-0.000525,0.000938,...,-0.000980,0.001735,0.007113,-0.015910,-0.008251,0.005404,0.001874,-0.012119,0.010584,0.016732


Split Data

In [14]:
X_train=features[:-500]
X_test=features[-500:]
y_train=Target[:-500]
y_test=Target[-500:]

# Modeling

# 1,try to predict the exact daily return(Regression Problem)
using train_predict pipeline to choose the best model

In [15]:
# TODO: Import two metrics from sklearn - fbeta_score and accuracy_score
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd

def train_predict(learner, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''

    
    results = {}
    learner = learner.fit(X_train,y_train)
    predictions_test = learner.predict(X_test)
    # TODO: Compute accuracy on test set
    results['R2_test'] = r2_score(y_test,predictions_test)
        
    # Return the results
    return results

In [149]:
from sklearn import linear_model
from sklearn import svm
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet

# TODO: Initialize the three models
clf_A = linear_model.Ridge (random_state=1)
clf_B = svm.SVR(kernel='linear')
clf_C = svm.SVR(kernel='rbf')
clf_D=AdaBoostRegressor(random_state=1)
clf_E=RandomForestRegressor(random_state=1)
clf_F = linear_model.Lasso(random_state=1)
clf_G =  ElasticNet(random_state=1)


# Collect results on the learners
results = {}
for clf in [clf_A, clf_B, clf_C, clf_D, clf_E,clf_F,clf_G]:
    clf_name = clf.__class__.__name__
    results[clf_name] = train_predict(clf, X_train, y_train, X_test, y_test)
    

In [150]:
print(results)

{'Ridge': {'R2_test': -0.063548623767675005}, 'AdaBoostRegressor': {'R2_test': -0.010078531724320383}, 'RandomForestRegressor': {'R2_test': -0.12367595592000424}, 'ElasticNet': {'R2_test': -0.0024767043136966205}, 'SVR': {'R2_test': -0.14439830203484694}, 'Lasso': {'R2_test': -0.0024767043136966205}}


In [131]:
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.cross_validation import ShuffleSplit
from sklearn import linear_model
import numpy as np

def fit_model(X, y):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    # For Balance Data
    cv_sets = ShuffleSplit(X.shape[0], n_iter = 10, test_size = 0.20, random_state = 0)
    
    
    regressor = linear_model.Ridge(random_state=0)

   
    params = {'alpha':range(145,155,1)}

    # TODO: Transform 'performance_metric' into a scoring function using 'make_scorer' 
    scoring_fnc = make_scorer(r2_score)

    # TODO: Create the grid search object
    grid =GridSearchCV(regressor, params,scoring_fnc,cv=cv_sets)
    
    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

In [132]:
model=fit_model(X_train, y_train)

In [133]:
print(model.get_params()['alpha'])

145


In [134]:
y_predict=model.predict(X_test)
print(r2_score(y_test,y_predict))

0.00358043329235


# 2,try to predict the the Movement of  daily return(Classification Problem)
using train_predict pipeline to choose the best model

transfer Target data to binary

In [ ]:
Target[Target > 0] = 1
Target[Target <= 0] = 0

Split Data

In [ ]:
X_train=features[:-500]
X_test=features[-500:]
y_train=Target[:-500]
y_test=Target[-500:]

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
reg= RandomForestClassifier(n_estimators=1000)
reg.fit(X_train, y_train)
pre=reg.predict(X_test)
result = accuracy_score(y_test,pre)
print(result)

0.498


In [20]:
# TODO: Import two metrics from sklearn - fbeta_score and accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score
import numpy as np
import pandas as pd

def train_predict(learner, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''

    
    results = {}
    learner = learner.fit(X_train,y_train)
    predictions_test = learner.predict(X_test)
    # TODO: Compute accuracy on test set
    results['accuracy_score'] = accuracy_score(y_test,predictions_test)
        
    # Return the results
    return results

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier

# TODO: Initialize the three models
clf_A = RandomForestClassifier(n_estimators=10)
clf_B = KNeighborsClassifier()
clf_C = SVC()
clf_D=AdaBoostClassifier()
clf_E=GradientBoostingClassifier(n_estimators=100)
clf_F = SGDClassifier(loss="hinge", penalty="l2")


# Collect results on the learners
results = {}
for clf in [clf_A, clf_B, clf_C, clf_D, clf_E,clf_F]:
    clf_name = clf.__class__.__name__
    results[clf_name] = train_predict(clf, X_train, y_train, X_test, y_test)
    

In [22]:
# when addFeatures(datasets,3,3,3)
print(results)

{'KNeighborsClassifier': {'accuracy_score': 0.504}, 'RandomForestClassifier': {'accuracy_score': 0.47999999999999998}, 'AdaBoostClassifier': {'accuracy_score': 0.47599999999999998}, 'SVC': {'accuracy_score': 0.51200000000000001}, 'SGDClassifier': {'accuracy_score': 0.49199999999999999}, 'GradientBoostingClassifier': {'accuracy_score': 0.51000000000000001}}


In [23]:
# when addFeatures(datasets,9,9,9)
print(results)

{'KNeighborsClassifier': {'accuracy_score': 0.504}, 'RandomForestClassifier': {'accuracy_score': 0.47999999999999998}, 'AdaBoostClassifier': {'accuracy_score': 0.47599999999999998}, 'SVC': {'accuracy_score': 0.51200000000000001}, 'SGDClassifier': {'accuracy_score': 0.49199999999999999}, 'GradientBoostingClassifier': {'accuracy_score': 0.51000000000000001}}


In [29]:
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import ShuffleSplit
import numpy as np

def fit_model(X, y):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    # For Balance Data
    cv_sets = ShuffleSplit(X.shape[0], n_iter = 10, test_size = 0.20, random_state = 0)
    
    
    regressor = SVC(random_state=0)

   
    params = {'C':[0.3,0.5,0.7,1.0,1.3,1.5,1.7,2.0],'degree':[2,3,4,5,6,7,8],'kernel':[ 'linear', 'poly', 'rbf', 'sigmoid']}

    # TODO: Transform 'performance_metric' into a scoring function using 'make_scorer' 
    scoring_fnc = make_scorer(accuracy_score)

    # TODO: Create the grid search object
    grid =GridSearchCV(regressor, params,scoring_fnc,cv=cv_sets)
    
    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

In [30]:
model=fit_model(X_train, y_train)

In [33]:
print(model.get_params()['C'])
print(model.get_params()['degree'])
print(model.get_params()['kernel'])


0.3
2
linear


In [34]:
y_predict=model.predict(X_test)
print(accuracy_score(y_test,y_predict))

0.512
